In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt

In [2]:
torch.manual_seed(42)

In [3]:
df = pd.read_csv("fmnist_small.csv")
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [4]:
#train test split
x = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [5]:
x_train,x_test,y_train,y_test = train_test_split(x,y , test_size=0.2 , random_state=42)

In [6]:
#scaling the features, basically we are normalizing the values to an smaller intervals (0-1)
x_train = x_train/255.0
x_test = x_test/255.0

In [7]:
#creating custom dataset
class CustomDataset(Dataset):

    def __init__(self,features,labels):
        
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels,dtype=torch.long)
    
    def __len__(self):
        return len(self.features)

    def __getitem__(self,index):
        return self.features[index],self.labels[index]

In [8]:
#create train_dataset object
train_dataset = CustomDataset(x_train,y_train)

In [9]:
len(train_dataset)

4800

In [10]:
#create train_dataset object
test_dataset = CustomDataset(x_test,y_test)

In [11]:
len(test_dataset)

1200

In [12]:
#creating train and test loader
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [13]:
#define nn class
class MyNN(nn.Module):
    
    def __init__(self,num_features):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(num_features,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,10)
        )
    def forward(self,x):
        return self.model(x)

In [14]:
#set learning rate and epochs
learning_rate = 0.1
epochs = 100

In [15]:
#instantiate the model
model = MyNN(x_train.shape[1])

#loss function
criterion = nn.CrossEntropyLoss()

#optimizer
optimizer = optim.SGD(model.parameters(),lr =learning_rate)

In [16]:
#training loop
for epoch in range(epochs):

    total_epoch_loss = 0

    for batch_features, batch_labels in train_loader:

        #forward pass
        outputs = model(batch_features)
        #loss calculation
        loss =criterion(outputs,batch_labels)
        #backpropagation
        optimizer.zero_grad()
        loss.backward()
        #update the parameters
        optimizer.step()

        total_epoch_loss = total_epoch_loss + loss.item()

    avg_loss = total_epoch_loss/len(train_loader)
    print(f"Epoch:{epoch +1} and Loss : {avg_loss}")
        

Epoch:1 and Loss : 1.3216368583838145
Epoch:2 and Loss : 0.7793365317583084
Epoch:3 and Loss : 0.6427524542808533
Epoch:4 and Loss : 0.5751657458146413
Epoch:5 and Loss : 0.5278772719701131
Epoch:6 and Loss : 0.49531099418799085
Epoch:7 and Loss : 0.4619268523653348
Epoch:8 and Loss : 0.43578990375002224
Epoch:9 and Loss : 0.41821308076381686
Epoch:10 and Loss : 0.39949775497118634
Epoch:11 and Loss : 0.38570138782262803
Epoch:12 and Loss : 0.3719716985026995
Epoch:13 and Loss : 0.3493794798851013
Epoch:14 and Loss : 0.3455629136164983
Epoch:15 and Loss : 0.31592130129535995
Epoch:16 and Loss : 0.3110544161498547
Epoch:17 and Loss : 0.2934306888779004
Epoch:18 and Loss : 0.2854089424510797
Epoch:19 and Loss : 0.2734348543733358
Epoch:20 and Loss : 0.2617666217436393
Epoch:21 and Loss : 0.26644358336925505
Epoch:22 and Loss : 0.24347751920421917
Epoch:23 and Loss : 0.23926382559041182
Epoch:24 and Loss : 0.225456866795818
Epoch:25 and Loss : 0.22396456686158975
Epoch:26 and Loss : 0.213

In [18]:
#set the model to eval mode
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [29]:
#evaluation 
total = 0
correct = 0
with torch.no_grad():
    for batch_features,batch_labels in test_loader:

        outputs = model(batch_features)
        _,predicted = torch.max(outputs,1)
        
        total = total + batch_features.shape[0]
        correct = correct + (predicted == batch_labels).sum().item() 
    
    accuracy = correct/total
    print(accuracy)

        

0.7966666666666666
